# 구글 플레이 스토어 리뷰 웹 크롤러
- Contributor: Tony Park
- References
    - Tistory Blog | https://heytech.tistory.com/
    - Github | https://github.com/park-gb
- Last Updated@2022-01-27

# 초기 설정

In [12]:
pip install beautifulsoup4


The following command must be run outside of the IPython shell:

    $ pip install beautifulsoup4

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [13]:
pip install selenium


The following command must be run outside of the IPython shell:

    $ pip install selenium

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [14]:
pip intall tqdm


The following command must be run outside of the IPython shell:

    $ pip intall tqdm

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [15]:
pip intall pandas


The following command must be run outside of the IPython shell:

    $ pip intall pandas

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


## 크롬 드라이버 세팅

In [2]:
chrome_driver = '../chromedriver.exe' # 파일 확장자 이름 표기
# chrome_driver = '../chromedriver' # 파일 확장자 이름 미표기

## 수집 앱 URL

In [3]:
URL = 'https://play.google.com/store/books/details/%ED%95%98%EC%99%84_%ED%95%98%EB%A7%88%ED%84%B0%EB%A9%B4_%EC%97%B4%EC%8B%AC%ED%9E%88_%EC%82%B4_%EB%BB%94%ED%96%88%EB%8B%A4?id=OOlXDwAAQBAJ'

# 패키지 import

In [4]:
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from time import sleep
import random
from tqdm.auto import tqdm, trange
import pandas as pd

# 무한 스크롤 함수

In [5]:
def scroll(modal):
    try:        
        # 스크롤 높이 받아오기
        last_height = driver.execute_script("return arguments[0].scrollHeight", modal)
        while True:
            pause_time = random.uniform(0.5, 0.8)
            # 최하단까지 스크롤
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", modal)
            # 페이지 로딩 대기
            time.sleep(pause_time)
            # 무한 스크롤 동작을 위해 살짝 위로 스크롤
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight-50);", modal)
            time.sleep(pause_time)
            # 스크롤 높이 새롭게 받아오기
            new_height = driver.execute_script("return arguments[0].scrollHeight", modal)
            try:
                # '더보기' 버튼 있을 경우 클릭
                all_review_button = driver.find_element_by_xpath('/html/body/div[1]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div[2]/div/span/span').click()
            except:
                # 스크롤 완료 경우
                if new_height == last_height:
                    print("스크롤 완료")
                    break
                last_height = new_height
                
    except Exception as e:
        print("에러 발생: ", e)

# 데이터 크롤링

In [5]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(chrome_driver, chrome_options = chrome_options)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [6]:
# 크롬 드라이버 세팅
driver = webdriver.Chrome(chrome_driver)
# 페이지 열기
driver.get(URL)
# 페이지 로딩 대기
wait = WebDriverWait(driver, 5)

# '리뷰 모두 보기' 버튼 렌더링 확인
all_review_button_xpath = '/html/body/c-wiz[2]/div/div/div[1]/div[2]/div/div[1]/c-wiz[3]/section/div/div/div[5]/div/div/button/span'
button_loading_wait = wait.until(EC.element_to_be_clickable((By.XPATH, all_review_button_xpath)))
# '리뷰 모두 보기' 버튼 클릭
driver.find_element_by_xpath(all_review_button_xpath).click()

# '리뷰 모두 보기' 페이지 렌더링 대기
all_review_page_xpath = '/html/body/div[4]/div[2]/div/div/div/div/div[2]'
page_loading_wait = wait.until(EC.element_to_be_clickable((By.XPATH, all_review_page_xpath)))

# 페이지 무한 스크롤 다운
modal = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='fysCi']")))
scroll(modal)

스크롤 완료


# HTML Parsing

In [7]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.addArguments("headless");
chrome_options.addArguments("no-sandbox");
chrome_options.addArguments("disable-dev-shm-usage");
driver = webdriver.Chrome(chrome_driver, chrome_options = chrome_options)

AttributeError: 'Options' object has no attribute 'addArguments'

In [8]:
# html parsing하기
html_source = driver.page_source
soup_source = BeautifulSoup(html_source, 'html.parser')

# HTML 데이터 저장

In [9]:
# html 데이터 저장
with open("../dataset/data_html.html", "w", encoding = 'utf-8') as file:
    file.write(str(soup_source))

## 데이터프레임 변환

In [10]:
# 리뷰 데이터 클래스 접근
review_source = soup_source.find_all(class_ = 'RHo1pe')
# 리뷰 데이터 저장용 배열
dataset = []
# 데이터 넘버링을 위한 변수
review_num = 0 
# 리뷰 1개씩 접근해 정보 추출
for review in tqdm(review_source):
    review_num+=1
    # 리뷰 등록일 데이터 추출
    date_full = review.find_all(class_ = 'bp9Aid')[0].text
    date_year = date_full[0:4] # 연도 데이터 추출
    # 해당 단어가 등장한 인덱스 추출
    year_index = date_full.find('년')
    month_index = date_full.find('월')
    day_index = date_full.find('일')
    
    date_month = str(int(date_full[year_index+1:month_index])) # 월(Month) 데이터 추출
    # 월 정보가 1자리의 경우 앞에 0 붙이기(e.g., 1월 -> 01월)
    if len(date_month) == 1:
        date_month = '0' + date_month
    
    date_day = str(int(date_full[month_index+1:day_index])) # 일(Day) 데이터 추출 
    # 일 정보가 1자리의 경우 앞에 0 붙여줌(e.g., 7일 -> 07일)
    if len(date_day) == 1:
        date_day = '0' + date_day
    
    # 리뷰 등록일 full version은 최종적으로 yyyymmdd 형태로 저장
    date_full = date_year + date_month + date_day
    user_name = review.find_all(class_ = 'X5PpBb')[0].text # 닉네임 데이터 추출
    rating = review.find_all(class_ = "iXRFPc")[0]['aria-label'][10] # 평점 데이터 추출
    content = review.find_all(class_ = 'h3YV2d')[0].text # 리뷰 데이터 추출

    data = {
        "id": review_num, 
        "date": date_full,
        "dateYear": date_year,
        "dateMonth": date_month,
        "dateDay": date_day,
        "rating": rating,
        "userName": user_name,
        "content": content
    }
    dataset.append(data)

## 데이터프레임 저장

In [11]:
df = pd.DataFrame(dataset)
df.to_csv('../dataset/review_dataset.csv', encoding = 'utf-8-sig') # csv 파일로 저장

# 리뷰 데이터 불러오기

In [12]:
# 저장한 리뷰 정보 불러오기
df = pd.read_csv('../dataset/review_dataset.csv', encoding = 'utf-8-sig')
df = df.drop(['Unnamed: 0'], axis = 1) # 불필요한 칼럼 삭제
df

,content,date,dateDay,dateMonth,dateYear,id,rating,userName
0,재미있게 읽었습니다.,20180705,5,7,2018,1,5,말랑제리
1,40을 바라보며 공감 공감 개공감~,20180917,17,9,2018,2,4,형진김
2,ㅋㅋㅋ 재밌다 이연정님 그건 인정합니다 ㅋㅋㅋ,20180812,12,8,2018,3,5,Google 사용자
3,틈틈히 가볍게 보기 좋은 책,20180719,19,7,2018,4,4,Minji Kim
4,공감 많이 됩니다 읽으면서 기뻤어요,20181012,12,10,2018,5,3,한한미정
5,대여가 없는게 아쉽네요,20180906,6,9,2018,6,3,나의봄
6,공감되는 부분이 너무 많아요,20180810,10,8,2018,7,5,Google 사용자
7,가볍게 쓰셨으나 깊이 생각하게 되었습니다,20180904,4,9,2018,8,5,최현수
8,경험과 현실에서 나오는 철학과 명언,20180827,27,8,2018,9,5,베지타맥스.vezitamax
9,그런 생각들이 요구될만한 가치의 시기이기도하네요.. 안타깝게도...,20180508,8,5,2018,10,2,김연경
